In [1]:
import cloudpickle as pickle
import os
from tqdm.auto import tqdm
import numpy as np
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [2]:
ckpt = '/home/sergei/ray_results/ve5_linear_tune_dimension_single_sample/main_fcn_638a7_00000_0__gin__fdim__choice=10_2020-12-06_17-02-06/checkpoint_0/checkpoint'

In [3]:
learner = pickle.load(open(ckpt, 'rb'))

Make environment VectorIncrement-v0 [<class 'encoder.observation_encoder.KerasEncoderWrapper'>] {}
Loading model /home/sergei/git/science/causality-disentanglement/encoder/encoders/encoder-config-ve5_linear-2c77a48c-fde6-11ea-96a3-00155d22e64a.pb
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Using device cpu


In [4]:
for _ in tqdm(range(100)):
    _ = learner._epoch()

In [5]:
obs = learner._context['obs']
print(obs.shape)
obs

torch.Size([1020, 100])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0112,  0.0291,  0.0252,  ...,  0.0018, -0.0158, -0.0216],
        [ 0.0237, -0.0147, -0.0213,  ..., -0.0267, -0.0087,  0.0208],
        ...,
        [ 0.4560, -0.3676,  0.4834,  ..., -0.8583, -0.0515, -0.1732],
        [ 0.4684, -0.3957,  0.5303,  ..., -0.8899, -0.0439, -0.2032],
        [ 0.4541, -0.3825,  0.4882,  ..., -0.9079, -0.0462, -0.1841]])

In [6]:
f = learner.decoder(obs)
f

tensor([[-0.0424,  0.5463,  1.2542,  ..., -0.9460,  0.0139, -1.4971],
        [-0.2312,  0.3278,  1.2723,  ..., -0.7337, -0.1673, -1.3224],
        [-0.0283,  0.4783,  1.0548,  ..., -0.8283,  0.0321, -1.2918],
        ...,
        [ 0.3474, -0.1727, -1.4063,  ...,  0.5062,  0.4508,  1.2214],
        [ 0.4315, -0.0849, -1.4462,  ...,  0.4304,  0.5447,  1.1857],
        [ 0.5546, -0.0025, -1.5715,  ...,  0.3792,  0.6483,  1.1978]],
       grad_fn=<NativeBatchNormBackward>)

In [7]:
np.mean(np.abs((learner.reconstructor(f) - obs).cpu().detach().numpy()))

0.3059333

In [8]:
learner.decoder

LinearDecoder(
  (fc): Linear(in_features=100, out_features=10, bias=False)
  (bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
)

In [9]:
adam = torch.optim.Adam(lr=1e-3, params=list(learner.decoder.parameters()) + list(learner.reconstructor.parameters()))

In [10]:
# m = torch.nn.Linear(in_features=5, out_features=5)
# adam = torch.optim.Adam(params=m.parameters())

In [17]:
pre_bn = learner.decoder.fc(obs)
post_bn = learner.decoder.bn(pre_bn)

In [33]:
torch.max(torch.abs((pre_bn - torch.mean(pre_bn, axis=0)) / torch.std(pre_bn, axis=0) - post_bn))

tensor(0.0013, grad_fn=<MaxBackward1>)

In [42]:
for _ in range(1000):
    adam.zero_grad()
    loss = torch.mean(torch.square(learner.reconstructor(learner.decoder(obs)) - obs))
#     loss = torch.mean(torch.square(obs - m(obs)))
    loss.backward()
    adam.step()
print(loss)

tensor(0.1096, grad_fn=<MeanBackward0>)


In [58]:
list(learner.reconstructor.parameters())

[Parameter containing:
 tensor([[ 2.3217e-01, -1.0968e-01, -1.7343e-01, -1.8227e-01,  2.2426e-01,
          -1.7311e-01, -4.2434e-01, -1.5738e-01],
         [-1.9565e-02, -1.0329e-01, -4.3503e-01,  4.2567e-01, -1.7511e-01,
           8.0593e-02,  1.9490e-01, -1.2746e-02],
         [-1.3838e-01, -3.1716e-01, -2.4874e-01, -3.5508e-02,  4.9626e-02,
           2.0345e-01, -2.9795e-01, -9.7920e-02],
         [-3.9401e-02,  2.9890e-01,  2.3588e-01, -2.6960e-01, -2.0583e-01,
          -1.2440e-01, -2.0585e-02,  1.2866e-01],
         [ 1.7982e-01,  2.6285e-01, -1.9551e-01, -4.7579e-02,  1.1886e-01,
          -1.9094e-01, -2.1975e-01,  2.2679e-01],
         [ 3.1186e-02,  9.5058e-02,  1.8701e-01,  1.3974e-01,  1.6782e-01,
          -1.3202e-01, -4.9181e-02,  2.0464e-01],
         [-2.0697e-01, -2.3234e-01, -1.4131e-01, -9.0053e-02, -1.0202e-03,
          -3.3696e-01, -1.5993e-01, -3.0467e-01],
         [-2.6094e-01,  2.0865e-01,  2.0310e-01, -1.6680e-01, -3.1896e-01,
          -1.9406e-01,  3.9